# 🎯 Professional-Grade Single-Player Re-Identification (Max Robustness)

This notebook implements a top-tier, robust pipeline for tracking a single player. It is designed to handle real-world challenges like camera motion, occlusions, and similar-looking players with maximum accuracy.

### Core Upgrades:
- **Detector**: **YOLOv8-Large** for high-accuracy bounding boxes.
- **Re-ID Model**: **OpenCLIP ViT-H-14 (Huge)** for the most powerful and discriminative visual features.
- **Robust Tracking Algorithm**:
  - **Camera Motion Compensation**: Actively calculates and compensates for camera pan/zoom to stabilize predictions.
  - **Attribute Gating**: Creates and matches a **Color Profile** of the target's uniform to prevent confusion with other teams.
  - **Enhanced Re-ID Memory**: Maintains a gallery of recent appearances for reliable re-identification after long occlusions.

In [ ]:
# STEP 1: Install Upgraded Dependencies
print("Installing and configuring the upgraded environment...")
!pip -q uninstall -y tensorflow tensorflow-hub numba onnxruntime onnxruntime-gpu opencv-python opencv-contrib-python opencv-python-headless torchaudio torchvision || true
!pip -q install "numpy~=2.0.0" "opencv-python-headless>=4.9.0"
!pip -q install "ultralytics>=8.0.0" "scipy>=1.10.0" "filterpy>=1.4.5"
!pip -q install "torch>=2.3.0" --index-url https://download.pytorch.org/whl/cu121
!pip -q install "open-clip-torch>=2.24.0"
!pip -q install "pillow>=10.0.0" "tqdm>=4.66.0"

print("\n--- Verifying Installations ---")
try:
    import numpy as np, cv2, torch, open_clip, ultralytics, scipy, filterpy
    from ultralytics import YOLO
    print(f"NumPy version: {np.__version__}")
    print(f"OpenCV version: {cv2.__version__}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"Ultralytics (YOLOv8) version: {ultralytics.__version__}")
    print("✅ All libraries loaded successfully!")
except ImportError as e:
    print(f"❌ Error during verification: {e}")

In [ ]:
# STEP 2: Initialize Upgraded Models
import torch
import open_clip
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
from typing import Optional

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

print("Loading YOLOv8-Large model...")
detector = YOLO('yolov8l.pt')
detector.to(device)

print("Loading OpenCLIP ViT-H/14 model (this may take a moment)...")
reid_model, _, reid_preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', device=device)
reid_model.eval()

def get_embedding(crop_bgr: np.ndarray) -> Optional[np.ndarray]:
    if crop_bgr.size == 0: return None
    try:
        rgb = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        img_tensor = reid_preprocess(pil_img).unsqueeze(0).to(device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            features = reid_model.encode_image(img_tensor)
            features /= features.norm(dim=-1, keepdim=True)
        return features.cpu().numpy().squeeze()
    except Exception as e:
        return None

def get_color_hist(crop_bgr: np.ndarray) -> Optional[np.ndarray]:
    if crop_bgr.size == 0: return None
    try:
        lab_crop = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2LAB)
        hist = cv2.calcHist([lab_crop], [1, 2], None, [32, 32], [0, 256, 0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()
    except Exception:
        return None

print("✅ All models initialized successfully!")

In [ ]:
# STEP 3: Define the Robust Professional Tracker
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment
from collections import deque

class RobustTrack:
    def __init__(self, bbox, track_id, embedding, color_hist):
        self.id = track_id
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,1], [0,0,0,1,0,0,0], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]])
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01
        self.kf.x[:4] = self.convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.hits = 1
        self.age = 0
        self.gallery = deque(maxlen=100)
        self.gallery.append(embedding)
        self.color_profile = color_hist

    def convert_bbox_to_z(self, bbox):
        w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        x, y = bbox[0] + w/2., bbox[1] + h/2.
        s, r = w * h, w / float(h) if h > 0 else 0
        return np.array([x, y, s, r]).reshape((4, 1))

    def predict(self, motion_matrix=None):
        if motion_matrix is not None:
            # Apply camera motion compensation
            pos = np.array([self.kf.x[0,0], self.kf.x[1,0], 1.0]).reshape(3,1)
            new_pos = motion_matrix @ pos
            self.kf.x[0,0] = new_pos[0,0]
            self.kf.x[1,0] = new_pos[1,0]
        self.kf.predict()
        self.age += 1
        self.time_since_update += 1
        return self.get_state()

    def update(self, bbox, embedding, color_hist):
        self.time_since_update = 0
        self.hits += 1
        self.kf.update(self.convert_bbox_to_z(bbox))
        self.gallery.append(embedding)
        # Update color profile with EMA
        self.color_profile = 0.9 * self.color_profile + 0.1 * color_hist

    def get_state(self):
        x, y, s, r = self.kf.x[:4].flatten()
        w = np.sqrt(s * r) if s * r > 0 else 0
        h = s / w if w > 0 else 0
        return np.array([x - w/2., y - h/2., x + w/2., y + h/2.])

class RobustTracker:
    def __init__(self, max_age=90, reid_threshold=0.5, color_threshold=0.7):
        self.max_age = max_age
        self.reid_threshold = reid_threshold
        self.color_threshold = color_threshold
        self.track = None # We only track one player

    def update(self, detections, embeddings, color_hists, motion_matrix=None):
        if self.track:
            self.track.predict(motion_matrix)

        if not self.track:
            if len(detections) > 0:
                areas = (detections[:, 2] - detections[:, 0]) * (detections[:, 3] - detections[:, 1])
                idx = np.argmax(areas)
                self.track = RobustTrack(detections[idx], 1, embeddings[idx], color_hists[idx])
        else:
            if len(detections) > 0:
                # Calculate combined similarity score
                gallery_sims = np.array([np.dot(g, e) for g in self.track.gallery for e in embeddings]).reshape(len(self.track.gallery), len(embeddings))
                max_gallery_sims = gallery_sims.max(axis=0)
                color_sims = np.array([cv2.compareHist(self.track.color_profile, h, cv2.HISTCMP_CORREL) for h in color_hists])
                
                combined_scores = 0.7 * max_gallery_sims + 0.3 * color_sims
                best_match_idx = np.argmax(combined_scores)

                if combined_scores[best_match_idx] > self.reid_threshold and color_sims[best_match_idx] > self.color_threshold:
                    self.track.update(detections[best_match_idx], embeddings[best_match_idx], color_hists[best_match_idx])
        
        if self.track and self.track.time_since_update > self.max_age:
            self.track = None

        if self.track and self.track.time_since_update == 0:
            return [self.track.get_state()]
        return []

print("✅ Robust professional tracker defined successfully!")

In [ ]:
# STEP 4: Define the Main Video Processing Function with Enhancements
from tqdm.notebook import tqdm

def process_video_robust(
    input_path: str, 
    output_path: str, 
    highlight_path: str = '/content/highlights.mp4'
):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_full = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    out_high = cv2.VideoWriter(highlight_path, fourcc, fps, (width, height))

    tracker = RobustTracker(max_age=int(fps*3), reid_threshold=0.4, color_threshold=0.6)
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    prev_gray = None
    prev_kps = None
    prev_des = None

    for frame_idx in tqdm(range(total_frames), desc='Processing Video'):
        ret, frame = cap.read()
        if not ret: break

        # --- Camera Motion Compensation ---
        motion_matrix = None
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kps, des = orb.detectAndCompute(gray, None)
        if prev_gray is not None and des is not None and prev_des is not None:
            matches = bf.match(prev_des, des)
            if len(matches) > 50:
                src_pts = np.float32([prev_kps[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
                dst_pts = np.float32([kps[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
                motion_matrix, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        prev_gray, prev_kps, prev_des = gray, kps, des

        # --- Detection and Feature Extraction ---
        results = detector(frame, classes=[0], verbose=False)
        detections = results[0].boxes.xyxy.cpu().numpy()
        
        valid_detections, embeddings, color_hists = [], [], []
        for i, (x1, y1, x2, y2) in enumerate(detections):
            crop = frame[int(y1):int(y2), int(x1):int(x2)]
            emb = get_embedding(crop)
            hist = get_color_hist(crop)
            if emb is not None and hist is not None:
                embeddings.append(emb)
                color_hists.append(hist)
                valid_detections.append(detections[i])
        
        detections = np.array(valid_detections)
        embeddings = np.array(embeddings)
        color_hists = np.array(color_hists)

        # --- Tracker Update ---
        tracked_bboxes = tracker.update(detections, embeddings, color_hists, motion_matrix)

        # --- Drawing ---
        if tracked_bboxes:
            for bbox in tracked_bboxes:
                x1, y1, x2, y2 = map(int, bbox)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(frame, "TARGET PLAYER", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                out_high.write(frame)
        
        out_full.write(frame)

    cap.release()
    out_full.release()
    out_high.release()
    print("\n--- Processing Complete ---")
    print(f"✅ Full video saved to: {output_path}")
    print(f"✅ Highlights video saved to: {highlight_path}")

In [ ]:
# STEP 5: Upload Video
from google.colab import files

print('📹 Upload your video file (.mp4, .mov, etc.)...')
uploaded_video = files.upload()

input_video_path = None
if uploaded_video:
    input_video_path = f'/content/{next(iter(uploaded_video))}'
    print(f"\n✅ Video ready: {input_video_path}")
else:
    print("\n❌ No video file was uploaded. Please run this cell again.")

In [ ]:
# STEP 6: Run Processing and Download Results

if input_video_path:
    output_video_path = '/content/tracked_output.mp4'
    highlight_video_path = '/content/highlights.mp4'
    
    process_video_robust(
        input_path=input_video_path,
        output_path=output_video_path,
        highlight_path=highlight_video_path
    )

    print("\n--- Download Your Files ---")
    try:
        files.download(highlight_video_path)
    except Exception as e:
        print(f"Could not trigger automatic download. Please use the file browser on the left. Error: {e}")
else:
    print("❌ Cannot run processing because no input video was found. Please upload a video in the cell above.")